## Download and unzip the dataset


In [ ]:
# download dataset from the UCI website
!curl -o uci-labelled-sentences.zip https://archive.ics.uci.edu/static/public/331/sentiment+labelled+sentences.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 84188    0 84188    0     0  78591      0 --:--:--  0:00:01 --:--:-- 78606


In [ ]:
# unzip dataset in Colab
!unzip uci-labelled-sentences.zip

Archive:  uci-labelled-sentences.zip
   creating: sentiment labelled sentences/
  inflating: sentiment labelled sentences/.DS_Store  
   creating: __MACOSX/
   creating: __MACOSX/sentiment labelled sentences/
  inflating: __MACOSX/sentiment labelled sentences/._.DS_Store  
  inflating: sentiment labelled sentences/amazon_cells_labelled.txt  
  inflating: sentiment labelled sentences/imdb_labelled.txt  
  inflating: __MACOSX/sentiment labelled sentences/._imdb_labelled.txt  
  inflating: sentiment labelled sentences/readme.txt  
  inflating: __MACOSX/sentiment labelled sentences/._readme.txt  
  inflating: sentiment labelled sentences/yelp_labelled.txt  
  inflating: __MACOSX/._sentiment labelled sentences  


## Import libraries

In [ ]:
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.callbacks import EarlyStopping

## Load the datasets into a Pandas dataframe

In [ ]:

df_list = []

# Yelp
df_yelp = pd.read_csv('sentiment labelled sentences/yelp_labelled.txt', names=['sentence', 'label'], sep='\t')
df_yelp['source'] = 'yelp'
df_list.append(df_yelp)

# Amazon
df_amazon = pd.read_csv('sentiment labelled sentences/amazon_cells_labelled.txt', names=['sentence', 'label'], sep='\t')
df_amazon['source'] = 'amazon'
df_list.append(df_amazon)

# IMDB
df_imdb = pd.read_csv('sentiment labelled sentences/imdb_labelled.txt', names=['sentence', 'label'], sep='\t')
df_imdb['source'] = 'imdb'
df_list.append(df_imdb)

# Concatenate the dataframes
df = pd.concat(df_list)

df.head()

,sentence,label,source
0,Wow... Loved this place.,1,yelp
1,Crust is not good.,0,yelp
2,Not tasty and the texture was just nasty.,0,yelp
3,Stopped by during the late May bank holiday of...,1,yelp
4,The selection on the menu was great and so wer...,1,yelp


## Tokenize and pad

In [ ]:
max_features = 2000
tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(df['sentence'].values)
X = tokenizer.texts_to_sequences(df['sentence'].values)
X = pad_sequences(X)
y = df['label'].values

## Split the dataset into train and test sets


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.12)

## Define the model

In [ ]:
def create_model(embed_dim = 64):
  model = Sequential()
  model.add(Embedding(max_features, embed_dim, input_length=X.shape[1]))
  model.add(LSTM(16))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

model = create_model()

## Train the model

In [ ]:
model.fit(X_train, y_train, epochs=6, batch_size=16, validation_data=(X_test, y_test),callbacks = [EarlyStopping(monitor='val_accuracy', min_delta=0.001, patience=2, verbose=1)])

Epoch 1/6
152/152 [==============================] - 28s 113ms/step - loss: 0.6455 - accuracy: 0.6390 - val_loss: 0.5621 - val_accuracy: 0.7303
Epoch 2/6
152/152 [==============================] - 8s 52ms/step - loss: 0.3657 - accuracy: 0.8536 - val_loss: 0.4575 - val_accuracy: 0.8061
Epoch 3/6
152/152 [==============================] - 7s 46ms/step - loss: 0.2075 - accuracy: 0.9256 - val_loss: 0.4321 - val_accuracy: 0.8424
Epoch 4/6
152/152 [==============================] - 6s 38ms/step - loss: 0.1320 - accuracy: 0.9586 - val_loss: 0.5095 - val_accuracy: 0.8212
Epoch 5/6
152/152 [==============================] - 6s 41ms/step - loss: 0.0918 - accuracy: 0.9739 - val_loss: 0.5437 - val_accuracy: 0.8182
Epoch 5: early stopping


## Save the model

In [ ]:
model.save("uci_sentimentanalysis.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


## Save the tokenizer

In [ ]:
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.DEFAULT_PROTOCOL)